In [ ]:
import sys
sys.path.insert(0, "..")
from src.config import PROJECT_ROOT, MODEL_SEED, LABELS, MODELS_TO_RUN
from src.variables import LLM_COLS, LAB_COLS, CODE_COLS, CATEGORY_COLS
from src.preprocessing import create_preprocessor
from src.models import get_models_and_search_space
from src.evaluation import youden_threshold, metrics_at_threshold, decision_curve
from src.feature_utils import (
    find_feature_list_file, read_feature_list, load_xy,
    load_feature_name_map, prettify_name, prettify_names,
    clean_ct_feature_name, inject_psych_scale_aliases, prettify_psych_name,
    FRIENDLY_OVERRIDES, LEVEL_MAP, to_bayes_space
)

import os, re, joblib, json, glob, warnings
import numpy as np
import pandas as pd
from pathlib import Path
from dataclasses import dataclass
from typing import List, Dict, Tuple

from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import (
    roc_auc_score, average_precision_score, brier_score_loss,
    precision_recall_fscore_support, accuracy_score, precision_score,
    recall_score, f1_score, roc_curve, confusion_matrix,
)
from sklearn.calibration import calibration_curve
from sklearn.pipeline import Pipeline
from sklearn.base import clone

from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical

import shap
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore", category=UserWarning)
plt.rcParams.update({"figure.dpi": 130, "axes.spines.top": False, "axes.spines.right": False})


In [ ]:
# ---------------------------
# ⚠️⚠️⚠️⚠️0) 경로/조합 설정⚠️⚠️⚠️⚠️
# ---------------------------
OUT_DIR = Path(
    str(PROJECT_ROOT / "results/new_analysis/260114_qwen/modeling/step2_modeling/simple_20")
)
FIG_DIR = OUT_DIR / "figures"
OUT_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

IMP_DIR = Path(
    str(PROJECT_ROOT / "data/processed_imp/260114_split_corr_LLM_ADER/imputation/simple_imput")
)
FS_DIR = Path(
    str(PROJECT_ROOT / "results/new_analysis/260114_qwen/Feature_Selection/simple_20/step2_FS")
)

# ⚠️Train/Test 파일 이름⚠️
DATA_FILES = {
    "label_30d": {
        "train": IMP_DIR / "simple_label_30d_train.csv",
        "test": IMP_DIR / "simple_label_30d_test.csv",
    },
    "label_60d": {
        "train": IMP_DIR / "simple_label_60d_train.csv",
        "test": IMP_DIR / "simple_label_60d_test.csv",
    },
    "label_90d": {
        "train": IMP_DIR / "simple_label_90d_train.csv",
        "test": IMP_DIR / "simple_label_90d_test.csv",
    },
    "label_180d": {
        "train": IMP_DIR / "simple_label_180d_train.csv",
        "test": IMP_DIR / "simple_label_180d_test.csv",
    },
    "label_365d": {
        "train": IMP_DIR / "simple_label_365d_train.csv",
        "test": IMP_DIR / "simple_label_365d_test.csv",
    },
}

# 당신의 탐색 결과를 반영한 권장조합 (필요시 수정)
BEST_COMBOS = {
    "label_30d": {"model": "rf", "feature_set": "All_Features"},
    "label_60d": {"model": "rf", "feature_set": "All_Features"},
    "label_90d": {"model": "rf", "feature_set": "All_Features"},
    "label_180d": {"model": "rf", "feature_set": "All_Features"},
    "label_365d": {"model": "rf", "feature_set": "All_Features"},
}

TARGET_COL_FALLBACK = "label"

# Variables imported from `src.variables`


In [ ]:
FEATURE_MAP_CSV = str(PROJECT_ROOT / "results/Feature_selection/w_LLM_v5/step2_FS/feature_summary.csv")
NAME_MAP = load_feature_name_map(FEATURE_MAP_CSV)

In [ ]:
# ---------------------------
# 4) Nested-CV on TRAIN → TEST 평가
# ---------------------------
def run_for_label(
    label: str, model_name: str, feature_set: str, random_state: int = 42
) -> Dict:
    print(f"\n==== {label} | model={model_name} | features={feature_set}====")
    # FS
    fs_file = find_feature_list_file(label, FS_DIR)
    fs_feats = read_feature_list(fs_file)

    # --- feature_set에 따라 LLM/Lab 컬럼 추가 ---
    fs_low = feature_set.lower()
    base_features = list(fs_feats)  # 원본 FS 리스트

    if fs_low == "all_features":
        # 1) All_Features: 불러온 FS + LLM
        # (정의: 불러온 feature 파일에서 LLM cols만 추가)
        final_features_list = [*base_features, *LLM_COLS]

    elif fs_low == "base_lab":
        # 2) Base_Lab: 불러온 FS 그대로
        # (정의: 그냥 불러온 feature 파일 사용)
        final_features_list = base_features

    elif fs_low == "base":
        # 3) Base: 불러온 FS에서 Lab 제외
        # (정의: 불러온 feature 파일에서 lab_cols에 해당하는 변수가 있으면 제외)
        lab_cols_set = set(LAB_COLS)
        final_features_list = [f for f in base_features if f not in lab_cols_set]

    elif fs_low == "base_llm":
        # 4) Base_LLM: 불러온 FS에서 Lab 제외 + LLM 추가
        # (정의: 불러온 feature 파일에서 lab_cols 제외하고 LLM cols 추가)
        lab_cols_set = set(LAB_COLS)
        base_only = [f for f in base_features if f not in lab_cols_set]
        final_features_list = [*base_only, *LLM_COLS]

    else:
        # 정의되지 않은 경우, 기본 'Base_Lab'으로 동작 (fs_feats 그대로 사용)
        print(
            f"[Warning] Unknown feature_set '{feature_set}'. Defaulting to 'Base_Lab' logic (using fs_feats as-is)."
        )
        final_features_list = base_features

    # 중복 제거 (순서 유지)
    features_final = list(dict.fromkeys(final_features_list))

    print(f"  [Info] Using {len(features_final)} features for '{feature_set}'.")

    # Load Train/Test
    train_path = DATA_FILES[label]["train"]
    test_path = DATA_FILES[label]["test"]
    X_tr, y_tr = load_xy(train_path, label, features_final)
    X_te, y_te = load_xy(test_path, label, features_final)

    # --- 전처리기: 카테고리/코드/수치 분리 (교집합만) ---
    # LLM/CATEGORY/CODE 리스트 중 실제 존재하는 것만 사용
    categorical_features = [c for c in CATEGORY_COLS if c in X_tr.columns]
    code_features = [c for c in CODE_COLS if c in X_tr.columns]
    # 수치형 = 나머지
    numeric_features = [
        c for c in X_tr.columns if c not in set(categorical_features + code_features)
    ]

    preprocessor = create_preprocessor(
        numeric_features=numeric_features,
        categorical_features=categorical_features,
        code_features=code_features,
    )

    # --- 모델 & 탐색공간 ---
    models = get_models_and_search_space()
    clf, grid = models[model_name.upper()]
    search_space = to_bayes_space(grid)

    # Pipeline (selector → SMOTE → estimator[+scaler])
    pipe = ImbPipeline(
        steps=[
            ("preprocessor", preprocessor),
            ("smote", SMOTE(sampling_strategy="minority", random_state=42)),
            ("clf", clf),
        ]
    )

    # Inner search
    inner = StratifiedKFold(n_splits=3, shuffle=True, random_state=random_state)
    search = BayesSearchCV(
        estimator=pipe,
        search_spaces=search_space,
        n_iter=50,  # code test : 5, Real test : 30 or 50
        cv=inner,
        scoring="roc_auc",
        n_jobs=-1,
        random_state=random_state,
        refit=True,
        verbose=0,
    )

    # Outer CV on Train (OOF)
    outer = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
    oof_prob = np.zeros(len(y_tr))
    oof_pred = np.zeros(len(y_tr), dtype=int)
    oof_fold = np.zeros(len(y_tr), dtype=int)

    fold_metrics = []
    for f, (tr_idx, va_idx) in enumerate(outer.split(X_tr, y_tr), start=1):
        X_tr_i, X_va_i = X_tr.iloc[tr_idx], X_tr.iloc[va_idx]
        y_tr_i, y_va_i = y_tr.iloc[tr_idx], y_tr.iloc[va_idx]
        search.fit(X_tr_i, y_tr_i)
        best = search.best_estimator_

        proba = best.predict_proba(X_va_i)[:, 1]
        pred = (proba >= 0.5).astype(int)

        oof_prob[va_idx] = proba
        oof_pred[va_idx] = pred
        oof_fold[va_idx] = f

        auc = roc_auc_score(y_va_i, proba)
        prauc = average_precision_score(y_va_i, proba)
        brier = brier_score_loss(y_va_i, proba)
        acc = accuracy_score(y_va_i, pred)
        prec, rec, f1, _ = precision_recall_fscore_support(
            y_va_i, pred, average="binary", zero_division=0
        )

        fold_metrics.append(
            {
                "label": label,
                "fold": f,
                "model": model_name,
                "auc": auc,
                "prauc": prauc,
                "brier": brier,
                "accuracy": acc,
                "precision": prec,
                "recall": rec,
                "f1": f1,
                "best_params": search.best_params_,
            }
        )
        print(f"  [Outer {f}] AUC={auc:.3f} | PR-AUC={prauc:.3f} | F1={f1:.3f}")

    # 저장: OOF & Fold metrics & summary
    oof_df = pd.DataFrame(
        {
            "label": label,
            "oof_fold": oof_fold,
            "y_true": y_tr.values,
            "y_prob": oof_prob,
            "y_pred": oof_pred,
        }
    )
    oof_df.to_csv(OUT_DIR / f"{label}__oof_predictions.csv", index=False)

    fold_df = pd.DataFrame(fold_metrics)
    fold_df.to_csv(OUT_DIR / f"{label}__train_fold_metrics.csv", index=False)

    train_summary = (
        fold_df.agg(
            {
                "auc": ["mean", "std"],
                "prauc": ["mean", "std"],
                "brier": ["mean", "std"],
                "accuracy": ["mean", "std"],
                "precision": ["mean", "std"],
                "recall": ["mean", "std"],
                "f1": ["mean", "std"],
            }
        )
        .T.reset_index()
        .rename(columns={"index": "metric"})
    )
    train_summary.insert(0, "split", "train_oof")
    train_summary.insert(0, "model", model_name)
    train_summary.insert(0, "label", label)

    # OOF mean±SD 요약 저장 (Nested-CV 결과 제시용)
    train_summary.to_csv(OUT_DIR / f"{label}__train_oof_summary.csv", index=False)

    # ⭐️⭐️⭐️ --- (A) OOF에서 Youden index 기반 최적 threshold --- ⭐️⭐️⭐️
    thr_oof, tpr_oof, fpr_oof = youden_threshold(
        oof_df["y_true"].values, oof_df["y_prob"].values
    )
    youden_oof = tpr_oof - fpr_oof
    oof_thr_metrics = metrics_at_threshold(
        oof_df["y_true"].values, oof_df["y_prob"].values, thr_oof
    )

    # 저장
    pd.DataFrame(
        [
            {
                "label": label,
                "model": model_name,
                "split": "train_oof",
                "youden_index": youden_oof,
                **oof_thr_metrics,
            }
        ]
    ).to_csv(OUT_DIR / f"{label}__youden_metrics_OOF.csv", index=False)
    # ⭐️⭐️⭐️ --- (A) OOF에서 Youden index 기반 최적 threshold --- ⭐️⭐️⭐️

    # OOF ROC / Calibration / DCA
    fpr, tpr, _ = roc_curve(oof_df["y_true"], oof_df["y_prob"])
    auc_oof = roc_auc_score(oof_df["y_true"], oof_df["y_prob"])
    plt.figure(figsize=(4.6, 4.2))
    plt.plot(fpr, tpr, lw=2, label=f"AUC={auc_oof:.3f}")
    plt.plot([0, 1], [0, 1], "--")
    # ⭐️⭐️⭐️Youden 최적점 표시 ⭐️⭐️⭐️
    plt.scatter(
        [fpr_oof], [tpr_oof], s=30, marker="o", label=f"Youden*: thr={thr_oof:.3f}"
    )
    # ⭐️⭐️⭐️Youden 최적점 표시 ⭐️⭐️⭐️
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(f"ROC (OOF) - {label} ({model_name})")
    plt.legend(loc="lower right")
    plt.tight_layout()
    plt.savefig(FIG_DIR / f"{label}__ROC_OOF.png")
    plt.close()

    pt, pp = calibration_curve(oof_df["y_true"], oof_df["y_prob"], n_bins=10)
    plt.figure(figsize=(4.6, 4.2))
    plt.plot(pp, pt, marker="o")
    plt.plot([0, 1], [0, 1], "--")
    plt.xlabel("Predicted")
    plt.ylabel("Observed")
    plt.title(
        f"Calibration (OOF) - {label} ({model_name})\nBrier={brier_score_loss(oof_df['y_true'], oof_df['y_prob']):.3f}"
    )
    plt.tight_layout()
    plt.savefig(FIG_DIR / f"{label}__Calibration_OOF.png")
    plt.close()

    dca_oof = decision_curve(oof_df["y_true"].values, oof_df["y_prob"].values)
    plt.figure(figsize=(4.6, 4.2))
    plt.plot(dca_oof["threshold"], dca_oof["net_benefit"], lw=2, label="Model")
    prev = oof_df["y_true"].mean()
    treat_all = dca_oof["threshold"].apply(
        lambda th: prev - (1 - prev) * (th / (1 - th))
    )
    plt.plot(dca_oof["threshold"], treat_all, "--", label="Treat-all")
    plt.plot(
        dca_oof["threshold"],
        np.zeros_like(dca_oof["threshold"]),
        "--",
        label="Treat-none",
    )
    plt.xlabel("Threshold")
    plt.ylabel("Net benefit")
    plt.title(f"DCA (OOF) - {label} ({model_name})")
    plt.legend()
    plt.tight_layout()
    plt.savefig(FIG_DIR / f"{label}__DCA_OOF.png")
    plt.close()
    dca_oof.to_csv(OUT_DIR / f"{label}__DCA_OOF.csv", index=False)

    # Train full로 재학습 → Test 평가
    print("  [Final fit on full Train] ...")
    search.fit(X_tr, y_tr)
    best_full = search.best_estimator_

    # --- ⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️ ---
    # --- ⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️ ---
    # (A) 최종 파이프라인 저장 (preprocessor + SMOTE + clf 모두 포함)
    model_path = OUT_DIR / f"{label}__best_model.pkl"
    joblib.dump(best_full, model_path)

    # (B) 이 라벨에서 실제로 사용한 "원본 컬럼명 리스트" 저장 (재현용)
    features_path = OUT_DIR / f"{label}__features_final.json"
    with open(features_path, "w", encoding="utf-8") as f:
        json.dump(features_final, f, ensure_ascii=False, indent=2)

    # (C) 전처리 후 피처명 저장 (원핫 포함된 최종 입력 피처명)
    preproc = best_full.named_steps["preprocessor"]
    try:
        raw_names = list(preproc.get_feature_names_out())
    except Exception:
        raw_names = [f"f{i}" for i in range(preproc.transform(X_tr.iloc[:1]).shape[1])]

    # 접두사 정리(선택): num__/cat__/code__ 및 num_/cat_/code_ 제거


    feat_names_clean = prettify_names(raw_names, NAME_MAP)
    featnames_path = OUT_DIR / f"{label}__transformed_feature_names.json"
    with open(featnames_path, "w", encoding="utf-8") as f:
        json.dump(feat_names_clean, f, ensure_ascii=False, indent=2)
    # --- ⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️ ---
    # --- ⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️ ---

    # --- ⭐️⭐️⭐️ (추가) 최종 모델의 best_params 저장 ⭐️⭐️⭐️ ---
    final_best_params = search.best_params_
    pd.DataFrame([{"label": label, "model": model_name, **final_best_params}]).to_csv(
        OUT_DIR / f"{label}__best_params_final.csv", index=False
    )

    prob_te = best_full.predict_proba(X_te)[:, 1]
    pred_te = (prob_te >= thr_oof).astype(int)

    # Test metrics
    auc_te = roc_auc_score(y_te, prob_te)
    pr_te = average_precision_score(y_te, prob_te)
    brier_te = brier_score_loss(y_te, prob_te)
    acc_te = accuracy_score(y_te, pred_te)
    prec_te, rec_te, f1_te, _ = precision_recall_fscore_support(
        y_te, pred_te, average="binary", zero_division=0
    )

    # --- ⭐️⭐️⭐️ Youden(Test) 먼저 계산 → 이후 test_metrics에 합치기 ⭐️⭐️⭐️ ---
    thr_te, tpr_te, fpr_te = youden_threshold(y_te.values, prob_te)
    youden_te = tpr_te - fpr_te
    test_thr_metrics = metrics_at_threshold(y_te.values, prob_te, thr_te)

    test_metrics = pd.DataFrame(
        [
            {
                "label": label,
                "model": model_name,
                "split": "test",
                "auc": auc_te,
                "prauc": pr_te,
                "brier": brier_te,
                "accuracy": acc_te,
                "precision": prec_te,
                "recall": rec_te,
                "f1": f1_te,
                "best_params": search.best_params_,
                "youden_index": youden_te,
                "thr_youden": thr_te,
                "sens_at_thr": test_thr_metrics["sensitivity"],
                "spec_at_thr": test_thr_metrics["specificity"],
                "ppv_at_thr": test_thr_metrics["ppv"],
                "npv_at_thr": test_thr_metrics["npv"],
            }
        ]
    )
    test_metrics.to_csv(OUT_DIR / f"{label}__test_metrics.csv", index=False)
    pd.DataFrame(
        [
            {
                "label": label,
                "model": model_name,
                "split": "test",
                "youden_index": youden_te,
                **test_thr_metrics,
            }
        ]
    ).to_csv(OUT_DIR / f"{label}__youden_metrics_Test.csv", index=False)

    # --- 변경: OOF 임계값(thr_oof) 기준을 메인으로 보고 ---
    # (1) OOF 임계값으로 Test 지표 계산 → 메인 보고
    test_thr_metrics_oof = metrics_at_threshold(y_te.values, prob_te, thr_oof)
    test_metrics = pd.DataFrame(
        [
            {
                "label": label,
                "model": model_name,
                "split": "test",
                "auc": auc_te,
                "prauc": pr_te,
                "brier": brier_te,
                # 분류 지표도 thr_oof 기준으로 재계산해 반영
                "accuracy": accuracy_score(y_te, (prob_te >= thr_oof).astype(int)),
                "precision": precision_score(
                    y_te, (prob_te >= thr_oof).astype(int), zero_division=0
                ),
                "recall": recall_score(y_te, (prob_te >= thr_oof).astype(int)),
                "f1": f1_score(y_te, (prob_te >= thr_oof).astype(int)),
                "best_params": search.best_params_,
                "thr_source": "oof",
                "thr_youden": thr_oof,
                "sens_at_thr": test_thr_metrics_oof["sensitivity"],
                "spec_at_thr": test_thr_metrics_oof["specificity"],
                "ppv_at_thr": test_thr_metrics_oof["ppv"],
                "npv_at_thr": test_thr_metrics_oof["npv"],
            }
        ]
    )
    test_metrics.to_csv(OUT_DIR / f"{label}__test_metrics.csv", index=False)

    # (선택) 참고용: Test에서 다시 구한 thr_te 결과는 별도 파일로만 저장 (부록)
    thr_te, tpr_te, fpr_te = youden_threshold(y_te.values, prob_te)
    youden_te = tpr_te - fpr_te
    test_thr_metrics_te = metrics_at_threshold(y_te.values, prob_te, thr_te)
    pd.DataFrame(
        [
            {
                "label": label,
                "model": model_name,
                "split": "test",
                "thr_source": "test",
                "thr_youden": thr_te,
                "youden_index": youden_te,
                **test_thr_metrics_te,
            }
        ]
    ).to_csv(OUT_DIR / f"{label}__youden_metrics_Test_fromTestThr.csv", index=False)

    # --- ⭐️⭐️⭐️(B) Test에서 Youden index 기반 최적 threshold --- ⭐️⭐️⭐️

    test_pred = pd.DataFrame(
        {"label": label, "y_true": y_te.values, "y_prob": prob_te, "y_pred": pred_te}
    )
    test_pred.to_csv(OUT_DIR / f"{label}__test_predictions.csv", index=False)

    # Test ROC / Calibration / DCA
    fpr, tpr, _ = roc_curve(y_te, prob_te)
    plt.figure(figsize=(4.6, 4.2))
    plt.plot(fpr, tpr, lw=2, label=f"AUC={auc_te:.3f}")
    plt.plot([0, 1], [0, 1], "--")
    # --- ⭐️⭐️⭐️ 추가 ⭐️⭐️⭐️

    def _tpr_fpr_at_thr(y_true, y_prob, thr):
        yhat = (y_prob >= thr).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_true, yhat).ravel()
        tpr = tp / (tp + fn + 1e-12)
        fpr = fp / (fp + tn + 1e-12)
        return tpr, fpr

    tpr_oof_on_test, fpr_oof_on_test = _tpr_fpr_at_thr(y_te.values, prob_te, thr_oof)
    plt.scatter(
        [fpr_oof_on_test],
        [tpr_oof_on_test],
        s=30,
        marker="o",
        label=f"Youden thr={thr_oof:.3f}",
    )
    # --- ⭐️⭐️⭐️ 추가 ⭐️⭐️⭐️
    plt.xlabel("FPR")
    plt.ylabel("TPR")
    plt.title(f"ROC (Test) - {label} ({model_name})")
    plt.legend(loc="lower right")
    plt.tight_layout()
    plt.savefig(FIG_DIR / f"{label}__ROC_Test.png")
    plt.close()

    pt, pp = calibration_curve(y_te, prob_te, n_bins=10)
    plt.figure(figsize=(4.6, 4.2))
    plt.plot(pp, pt, marker="o")
    plt.plot([0, 1], [0, 1], "--")
    plt.xlabel("Predicted")
    plt.ylabel("Observed")
    plt.title(f"Calibration (Test) - {label} ({model_name})\nBrier={brier_te:.3f}")
    plt.tight_layout()
    plt.savefig(FIG_DIR / f"{label}__Calibration_Test.png")
    plt.close()

    dca_te = decision_curve(y_te.values, prob_te)
    plt.figure(figsize=(4.6, 4.2))
    plt.plot(dca_te["threshold"], dca_te["net_benefit"], lw=2, label="Model")
    prev = y_te.mean()
    treat_all = dca_te["threshold"].apply(
        lambda th: prev - (1 - prev) * (th / (1 - th))
    )
    plt.plot(dca_te["threshold"], treat_all, "--", label="Treat-all")
    plt.plot(
        dca_te["threshold"],
        np.zeros_like(dca_te["threshold"]),
        "--",
        label="Treat-none",
    )
    plt.xlabel("Threshold")
    plt.ylabel("Net benefit")
    plt.title(f"DCA (Test) - {label} ({model_name})")
    plt.legend()
    plt.tight_layout()
    plt.savefig(FIG_DIR / f"{label}__DCA_Test.png")
    plt.close()
    dca_te.to_csv(OUT_DIR / f"{label}__DCA_Test.csv", index=False)

    # SHAP (full Train 재학습 모델 기준, Test 특성 사용)
    print("  [SHAP] compute on Test ...")

    preproc = best_full.named_steps["preprocessor"]
    clf_final = best_full.named_steps["clf"]

    # 1) 전처리 후 데이터 준비
    rng = np.random.RandomState(42)
    bg_idx = rng.choice(len(X_tr), size=min(200, len(X_tr)), replace=False)
    X_bg_t = preproc.transform(X_tr.iloc[bg_idx])
    X_te_t = preproc.transform(X_te)

    # ⭐️⛔️⭐️⛔️⭐️⛔️ 2) 전처리 후 피처 이름 + 접두사 정리 ⭐️⛔️⭐️⛔️⭐️⛔️⭐️⛔️
    # try:
    #     raw_names = list(preproc.get_feature_names_out())
    # except Exception:
    #     raw_names = [f"f{i}" for i in range(X_te_t.shape[1])]
    # 2) 전처리 후 피처 이름
    try:
        raw_names = list(preproc.get_feature_names_out())
    except Exception:
        raw_names = [f"f{i}" for i in range(X_te_t.shape[1])]
    # ⭐️⛔️⭐️⛔️⭐️⛔️ ⭐️⛔️⭐️⛔️⭐️⛔️⭐️⛔️


    # ⭐️⛔️⭐️⛔️⭐️⛔️ ⭐️⛔️⭐️⛔️⭐️⛔️⭐️⛔️
    feat_names = prettify_names(raw_names, NAME_MAP)
    # ⭐️⛔️⭐️⛔️⭐️⛔️ ⭐️⛔️⭐️⛔️⭐️⛔️⭐️⛔️

    # 3) 모델 타입별 Explainer (배경 데이터 제공 → interventional 경고 해소)
    try:
        if isinstance(
            clf_final,
            (XGBClassifier, LGBMClassifier, CatBoostClassifier, RandomForestClassifier),
        ):
            explainer = shap.TreeExplainer(
                clf_final,
                data=X_bg_t,
                feature_perturbation="interventional",
                model_output="probability",
            )
            shap_expl = explainer(X_te_t)  # Explanation
        elif isinstance(clf_final, LogisticRegression):
            explainer = shap.LinearExplainer(clf_final, X_bg_t)
            shap_expl = explainer(X_te_t)  # ndarray 또는 Explanation
            if not isinstance(shap_expl, shap.Explanation):
                shap_expl = shap.Explanation(
                    values=shap_expl,
                    base_values=np.zeros(X_te_t.shape[0]),
                    data=X_te_t,
                    feature_names=feat_names,
                )
        else:
            explainer = shap.Explainer(clf_final, X_bg_t)
            shap_expl = explainer(X_te_t)
    except Exception:
        explainer = shap.Explainer(clf_final, X_bg_t)
        shap_expl = explainer(X_te_t)

    # 4) (중요) 다중 출력이면 양성 클래스(= index 1)만 선택 → beeswarm 오류 해결
    try:
        vals = shap_expl.values
        if getattr(vals, "ndim", 2) == 3:
            # (n, p, outputs) → outputs=1 선택
            shap_expl = shap_expl[:, :, 1]
    except Exception:
        pass

    # 5) 정리된 피처명 주입
    try:
        shap_expl.feature_names = feat_names
    except Exception:
        pass

    # 6) bar에서 “sum of other features” 제거: 상위 K개만 남긴 Explanation 생성
    K = 20
    # mean(|SHAP|) 기준 상위 K 인덱스
    mean_abs = np.abs(np.array(shap_expl.values)).mean(axis=0)
    topk_idx = np.argsort(mean_abs)[::-1][:K]
    shap_top = shap_expl[:, topk_idx]  # Explanation 슬라이싱 지원

    # 7) 시각화 (공식 API 사용)
    plt.figure()
    shap.plots.beeswarm(
        shap_expl,
        max_display=K,
        show=False,
        group_remaining_features=False,
    )
    plt.title(f"SHAP Summary (Test) - {label} ({model_name})")
    plt.tight_layout()
    plt.savefig(FIG_DIR / f"{label}__SHAP_summary_Test.png")
    plt.close()

    plt.figure()

    shap.plots.bar(
        shap_top, max_display=K, show=False
    )  # group_remaining_features=False
    plt.title(f"SHAP Top Features (Test) - {label} ({model_name})")
    plt.tight_layout()
    plt.savefig(FIG_DIR / f"{label}__SHAP_bar_Test.png")
    plt.close()

    # =============================
    # 🔽 [NEW] SHAP 값을 CSV로 저장
    # =============================
    try:
        # ---- 기본 배열/이름 준비
        shap_vals = np.asarray(shap_expl.values)  # (n_test, n_features)
        n_te, p = shap_vals.shape
        feat_names_arr = np.array(feat_names[:p])  # 안전 슬라이스
        row_ids = getattr(X_te, "index", np.arange(n_te))

        # ---- (1) 변수별 요약: 중요도 및 방향성
        mean_abs = np.abs(shap_vals).mean(axis=0)
        mean_val = shap_vals.mean(axis=0)
        std_val = shap_vals.std(axis=0)
        med_abs = np.median(np.abs(shap_vals), axis=0)
        q1_abs = np.quantile(np.abs(shap_vals), 0.25, axis=0)
        q3_abs = np.quantile(np.abs(shap_vals), 0.75, axis=0)
        nonzero_ratio = np.count_nonzero(shap_vals, axis=0) / float(n_te)

        fi_df = (
            pd.DataFrame(
                {
                    "label": label,
                    "model": model_name,
                    "feature": feat_names_arr,
                    "mean_abs_shap": mean_abs,
                    "mean_shap": mean_val,
                    "std_shap": std_val,
                    "median_abs_shap": med_abs,
                    "q1_abs_shap": q1_abs,
                    "q3_abs_shap": q3_abs,
                    "nonzero_ratio": nonzero_ratio,
                }
            )
            .sort_values("mean_abs_shap", ascending=False)
            .reset_index(drop=True)
        )
        fi_df.insert(0, "rank", np.arange(1, len(fi_df) + 1))
        fi_df.to_csv(
            OUT_DIR / f"{label}__SHAP_feature_importance_Test.csv", index=False
        )

        # ---- (2) 상위 K개만 별도 저장 (그림 max_display와 통일)
        topk = fi_df.head(K).copy()
        topk.to_csv(OUT_DIR / f"{label}__SHAP_top{K}_Test.csv", index=False)

        # ---- (3) 환자×변수 wide 매트릭스
        shap_wide = pd.DataFrame(shap_vals, index=row_ids, columns=feat_names_arr)
        shap_wide.index.name = "row_id"
        shap_wide.to_csv(OUT_DIR / f"{label}__SHAP_matrix_Test.csv")

        # ✅ (추가) 테스트 데이터(전처리 후 feature 값) wide matrix
        X_te_wide = pd.DataFrame(X_te_t, index=row_ids, columns=feat_names_arr)
        X_te_wide.index.name = "row_id"

        # ---- (4) long 포맷 (row_id, feature, shap_value, y_true, y_prob)
        test_prob_series = pd.Series(prob_te, index=row_ids, name="y_prob")
        test_true_series = pd.Series(y_te.values, index=row_ids, name="y_true")

        shap_long = shap_wide.reset_index().melt(
            id_vars=["row_id"], var_name="feature", value_name="shap_value"
        )

        # ✅ (추가) feature_value 컬럼 병합
        feat_long = X_te_wide.reset_index().melt(
            id_vars=["row_id"], var_name="feature", value_name="feature_value"
        )
        shap_long = shap_long.merge(feat_long, on=["row_id", "feature"], how="left")

        shap_long = shap_long.merge(
            test_true_series.reset_index(), on="row_id", how="left"
        ).merge(test_prob_series.reset_index(), on="row_id", how="left")
        shap_long.insert(0, "label", label)
        shap_long.insert(1, "model", model_name)
        shap_long.to_csv(OUT_DIR / f"{label}__SHAP_long_Test.csv", index=False)

        print(
            f"  [SHAP] CSV saved: "
            f"{label}__SHAP_feature_importance_Test.csv, "
            f"{label}__SHAP_top{K}_Test.csv, "
            f"{label}__SHAP_matrix_Test.csv, "
            f"{label}__SHAP_long_Test.csv"
        )
    except Exception as e:
        print(f"[WARN] Failed to write SHAP CSVs: {e}")

    # 통합 요약 반환
    return {
        "label": label,
        "model": model_name,
        "train_oof_summary_path": str(
            (OUT_DIR / f"{label}__train_fold_metrics.csv").resolve()
        ),
        "test_metrics_path": str((OUT_DIR / f"{label}__test_metrics.csv").resolve()),
        "oof_pred_path": str((OUT_DIR / f"{label}__oof_predictions.csv").resolve()),
        "test_pred_path": str((OUT_DIR / f"{label}__test_predictions.csv").resolve()),
    }


In [ ]:
def main():
    manifest = []
    for label, cfg in BEST_COMBOS.items():
        res = run_for_label(
            label=label,
            model_name=cfg["model"],
            feature_set=cfg["feature_set"],
            random_state=42,
        )
        manifest.append(res)
    pd.DataFrame(manifest).to_csv(OUT_DIR / "final_runs_manifest.csv", index=False)
    print("\nAll done. Results saved to:", OUT_DIR)


if __name__ == "__main__":
    main()